In [1057]:
# TASK DEFINITIONS
tasks = {
    # Custom Actions
    "brush teeth": {
        "needs": {},  # Requires 1 unit of water
        "depletes": {"toothpaste": 1},  # Depletes 1 unit of toothpaste
        "place": "bathroom",  # Needs to be in the bathroom
        "cost": 15,  # Cost of brushing teeth
    },
    "drink water": {
        "needs": {"water": 1},  # Requires 1 unit of water
        "depletes": {"water": 1},  # Depletes 1 unit of water
        "place": "kitchen",
        "cost": 5,
    },
    "clean dish": {
        "needs": {"soap": 1},  # Requires 1 unit of soap
        "depletes": {"soap": 1},  # Depletes 1 unit of soap
        "place": "kitchen",
        "cost": 12,
    },
    # Buy actions with costs
    "buy water": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": "store",
        "cost": 20,
    },
    "buy toothpaste": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": "store",
        "cost": 10,
    },
    "buy soap": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": "store",
        "cost": 3,
    },

    # Refill actions with costs
    "refill toothpaste": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": "storage",
        "cost": 6,
    },
    "refill soap": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": "storage",
        "cost": 1,
    },

    # INTERFACE
    "go to": {
        "needs": {},  # No specific resource needs  
        "depletes": {},  # No resources are depleted
        "place": {},
        "cost": 5,
    },
    "check": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": {},
        "cost": 3,
    },
    "check inventory": {
        "needs": {},  # No specific resource needs
        "depletes": {},  # No resources are depleted
        "place": {},
        "cost": 1,
    }
    
}

In [1058]:
# STORAGES
main_stock = {}  # Main stock for refilling room-specific stocks
room_stocks = {  # Dictionary to hold stocks for each room
    "bedroom" : {"water": 0, "bed": 1},
    "kitchen": {"water": 0, "soap": 1},
    "bathroom": {"water": 0, "toothpaste": 0},
    "store": {"water": 100, "soap": 100, "toothpaste": 100},  # Store stock to buy resources
}

In [1059]:
# CHECK LEVELS OF RESOURCE
def check_needs(task):
    return tasks.get(task, {}).get("needs", {})

# Method to check what a task depletes
def check_depletes(task):
    return tasks.get(task, {}).get("depletes", {})

def check_place(task):
    return tasks.get(task, {}).get("place", "")

def check_main_stock(resource):
    return main_stock.get(resource, 0)

# Method to check stock in a specific room
def check_room_stock(room, resource):
    return room_stocks.get(room, {}).get(resource, 0)

In [1060]:
# INITIALIZE PLAYER AND GAME
player_location = "bedroom"  # Starting location
total_cost = 0  # Track total cost

In [1061]:
# Initialize main stock with resources
def init_main_stock(resources):
    global main_stock
    main_stock = resources.copy()

In [1062]:
# Method to refill resources in a room from the main stock
def refill_room(room, resource, X):
    global total_cost, main_stock
    if main_stock.get(resource, 0) >= X:
        room_stocks[room][resource] += X
        main_stock[resource] -= X
        print(f"Refilled {X} {resource}(s) in the {room}. Main stock left: {main_stock[resource]}.")
        return True
    else:
        print(f"Not enough {resource} in the main stock to refill {room}. Needed: {X}, Available: {main_stock.get(resource, 0)}.")
        return False

# Method to buy resources from the store to replenish the main stock
def buy_resource(resource, X):
    global total_cost, room_stocks, main_stock
    if room_stocks["store"].get(resource, 0) >= X:
        room_stocks["store"][resource] -= X
        main_stock[resource] = main_stock.get(resource, 0) + X
        print(f"Bought {X} {resource}(s) from the store. ")
        return True
    else:
        print(f"Store doesn't have enough {resource}. Needed: {X}, Available: {room_stocks['store'].get(resource, 0)}.")
        return False

# Method to change location
def goto(place):
    global player_location, total_cost
    if player_location != place:
        player_location = place
        print(f"Moved to {place}.")
    else:
        print(f"Already at {place}.")
    
# Method to check current location
def where():
    return player_location

In [1063]:
def check_task_exist(task):
    print("Now executing", task)
    # go to storage
    if 'go to' in task:
         return True, task.split()[0]+ ' ' + task.split()[1]
    
    # check inventory for water
    if 'check inventory' in task:
         return True, task.split()[0] + ' ' + task.split()[1]
    
    # check kitchen for soap
    if 'check' in task:
         return True, task.split()[0]
    
    # Buy 5 apple
    if 'buy' in task:
        task = task.split()[0] + ' ' + task.split()[2]

    # Refill kitchen with 6 water
    if 'refill' in task:
        task = task.split()[0] + ' ' + task.split()[-1]
        
    if task not in tasks:
        print(f"Task '{task}' not recognized.")
        return False, task
    else:
        return True, task

In [1064]:
# Function to execute tasks

def execute_tasks(task_list):
    global total_cost
    total_cost = 0

    for task in task_list:
        task = task.lower()
        quant = False
        # Check if task exists
        bool_key = check_task_exist(task) # (Task valid bool, viable task key)
        if not bool_key[0]:
            return "Failure"
        
        elif 'check inventory' in task:
            resource = task.split()[3]
            print(check_main_stock(resource))

        elif 'check' in task:
            room = task.split()[1]
            resource = task.split()[3]
            print(check_room_stock(room, resource))

        elif 'go to' in task or 'Go to' in task:
            place = task.split()[2]
            goto(place)
        else:
            place = check_place(bool_key[1])
        
            # Now process place conditioned tasks
            if where() != place:
                print(f'Must be at {place}. You are at {where()}.')
                return "Failure"


            elif 'buy' in task:
                resource = task.split()[2]
                quantity = int(task.split()[1])
                buy_success = buy_resource(resource, quantity)
                if not buy_success:
                    return f"Failure, buying {resource} failed. Walmart doesnt have enough {resource}."
                quant = True

            elif 'refill' in task:
                room = task.split()[1]
                resource = task.split()[4]
                quantity = int(task.split()[3])
                refill_success = refill_room(room, resource, quantity)
                if not refill_success:
                    return f"Failure, refilling {resource} in {room} failed"
                quant = True
            else:
                # Check if required resources are available in the current room
                needs = check_needs(task)
                for resource, amount in needs.items():
                    if check_room_stock(where(), resource) < amount:
                        print(f"Not enough {resource} in {where()} to complete {task}. Needed: {amount}, Available: {check_room_stock(where(), resource)}.")
                        return "Failure"
                
                # Deplete resources from the room stock
                depletes = check_depletes(task)
                for resource, amount in depletes.items():
                    if check_room_stock(where(), resource) >= amount:
                        room_stocks[where()][resource] -= amount
                        print(f"Depleted {amount} {resource}(s) in {where()} for {task}. Remaining: {room_stocks[where()][resource]}")
                    else:
                        print(f"Not enough {resource} in {where()} to complete {task}. Needed: {amount}, Available: {check_room_stock(where(), resource)}.")
                        return "Failure"

        # Add task cost to total cost
        task_cost = tasks[bool_key[1]]["cost"]
        if quant == True:
            task_cost = quantity * task_cost
        total_cost += task_cost
        print(f"Executed {task} for cost {task_cost}. New total cost: {total_cost}")
    
    print(f"All tasks executed successfully. Total cost: {total_cost}")
    return "Success"

In [1065]:
def prompt_maker(task_list):
    print("WELCOME TO THE GAME")
    initial_prompt = f"You are standing in the {where()}. The home has: \n"
    # TODO: specify amount needed for each task
    
    # Give the agent rooms and their stock counts.
    room_str = ''
    for key, sub_dict in room_stocks.items():
        # Create a list of "value subkey" pairs
        sub_items = [f"{value} {sub_key}" for sub_key, value in sub_dict.items()]
        
        # Join the items with commas and "and" for the last item
        sub_items_joined = ", ".join(sub_items[:-1]) + " and " + sub_items[-1]
        
        # Print the sentence for each outer dictionary key
        room_str +=(f"A {key} with {sub_items_joined}. \n")
    initial_prompt += room_str
    initial_prompt += f"Your home's storage has: \n"

    stock_str = ', '.join([f"{value} {key}" for key, value in main_stock.items()])
    initial_prompt += f"{stock_str}. \n"

    # tasks
    initial_prompt += f"\n Your tasks are: \n"
    for task in task_list:
        initial_prompt += f"Task: {task} \n"

    # final instructions
    initial_prompt += f"\n These tasks are given in a random order.Your job is to rearrange these tasks in the correct order. You can also add go to, buy, and refill commands.\n Be careful about the given resource contraints and where you are. \n You can add refill, buy, and go to commands at a cost. \nFor example, the costs of buying one item are proportional to real world costs."
    initial_prompt += f"\n \n You can buy items from the store to restock your pantry by saying: \"buy <amount> <item>\" You must be in the store to buy items\n"
    initial_prompt += f"You can refill any room with items in your storage by saying: \"refill <room_name> with <amount> <item>\" You must be in the storage to refill items\n"
    initial_prompt += f"Remember to say \"Done\" when you are finished. \n"

    
    return initial_prompt

In [1066]:
# Initialize storage/pantry with resources
# Easy Example
init_main_stock({"water": 5, "toothpaste": 0})

# Define a list of tasks to execute
task_list = ["brush teeth"]

print(prompt_maker(task_list))

candidate_task_list = [
    "go to store",
    "buy 3 toothpaste",
    "go to storage",
    "refill bathroom with 1 toothpaste",
    "go to bathroom",
    "brush teeth"
]
# result = execute_tasks(task_list)
result_correct = execute_tasks(candidate_task_list)
# print(result)
print(result_correct)

# Graph 
# Node = task, edge = dependency between two tasks. (source = precondition task and target = task)
# 1) get naive decomp
# 2) build graph
# 3) fix the graph
# 4) output new decomp

# edge = call_LLM(task, task) => dependency between two tasks? - n^2
# directed graph of preconditions and test it against a dynamic knowledge base
# to make edits, search task list space for nighbors.

# explore symbolic planner and/or planning schema from code LLM.



WELCOME TO THE GAME
You are standing in the bedroom. The home has: 
A bedroom with 0 water and 1 bed. 
A kitchen with 0 water and 1 soap. 
A bathroom with 0 water and 0 toothpaste. 
A store with 100 water, 100 soap and 100 toothpaste. 
Your home's storage has: 
5 water, 0 toothpaste. 

 Your tasks are: 
Task: brush teeth 

 These tasks are given in a random order.Your job is to rearrange these tasks in the correct order. You can also add go to, buy, and refill commands.
 Be careful about the given resource contraints and where you are. 
 You can add refill, buy, and go to commands at a cost. 
For example, the costs of buying one item are proportional to real world costs.
 
 You can buy items from the store to restock your pantry by saying: "buy <amount> <item>" You must be in the store to buy items
You can refill any room with items in your storage by saying: "refill <room_name> with <amount> <item>" You must be in the storage to refill items
Remember to say "Done" when you are finishe

In [1067]:
# Harder Example INCOORECT

init_main_stock({"water": 5, "toothpaste": 0, "soap": 1})
goto("bedroom")

task_list = ["Clean 3 dishes"]
print(prompt_maker(task_list))

candidate_task_list = [
    "go to kitchen",
    "Clean dish",
    "clean dish",
    "Clean dish"
]
execute_tasks(candidate_task_list)

Moved to bedroom.
WELCOME TO THE GAME
You are standing in the bedroom. The home has: 
A bedroom with 0 water and 1 bed. 
A kitchen with 0 water and 1 soap. 
A bathroom with 0 water and 0 toothpaste. 
A store with 100 water, 100 soap and 97 toothpaste. 
Your home's storage has: 
5 water, 0 toothpaste, 1 soap. 

 Your tasks are: 
Task: Clean 3 dishes 

 These tasks are given in a random order.Your job is to rearrange these tasks in the correct order. You can also add go to, buy, and refill commands.
 Be careful about the given resource contraints and where you are. 
 You can add refill, buy, and go to commands at a cost. 
For example, the costs of buying one item are proportional to real world costs.
 
 You can buy items from the store to restock your pantry by saying: "buy <amount> <item>" You must be in the store to buy items
You can refill any room with items in your storage by saying: "refill <room_name> with <amount> <item>" You must be in the storage to refill items
Remember to say

'Failure'

In [1068]:
# Harder Example SOLVED

init_main_stock({"water": 5, "toothpaste": 0, "soap": 1})
goto("bedroom")
room_stocks["kitchen"]["soap"] = 1

task_list = ["Clean 3 dishes"]
print(prompt_maker(task_list))

candidate_task_list = [
    "go to store",
    "Buy 1 soap",
    "go to storage",
    "Refill kitchen with 2 soap",
    "go to kitchen",
    "Clean dish",
    "clean dish",
    "Clean dish"
]
execute_tasks(candidate_task_list)

Moved to bedroom.
WELCOME TO THE GAME
You are standing in the bedroom. The home has: 
A bedroom with 0 water and 1 bed. 
A kitchen with 0 water and 1 soap. 
A bathroom with 0 water and 0 toothpaste. 
A store with 100 water, 100 soap and 97 toothpaste. 
Your home's storage has: 
5 water, 0 toothpaste, 1 soap. 

 Your tasks are: 
Task: Clean 3 dishes 

 These tasks are given in a random order.Your job is to rearrange these tasks in the correct order. You can also add go to, buy, and refill commands.
 Be careful about the given resource contraints and where you are. 
 You can add refill, buy, and go to commands at a cost. 
For example, the costs of buying one item are proportional to real world costs.
 
 You can buy items from the store to restock your pantry by saying: "buy <amount> <item>" You must be in the store to buy items
You can refill any room with items in your storage by saying: "refill <room_name> with <amount> <item>" You must be in the storage to refill items
Remember to say

'Success'

In [1069]:
# Harder Example SOLVED

init_main_stock({"water": 5, "toothpaste": 0, "soap": 1})
goto("bedroom")
room_stocks["kitchen"]["soap"] = 1

task_list = ["Clean 3 dishes"]
print(prompt_maker(task_list))

candidate_task_list = [
    "go to kitchen",
    "clean dish",
    "go to storage",
    "refill kitchen with 1 soap",
    "go to kitchen",
    "clean dish",
    "go to store",
    "buy 1 soap",
    "go to storage",
    "refill kitchen with 1 soap",
    "go to kitchen",
    "clean dish"
]
execute_tasks(candidate_task_list)

Moved to bedroom.
WELCOME TO THE GAME
You are standing in the bedroom. The home has: 
A bedroom with 0 water and 1 bed. 
A kitchen with 0 water and 1 soap. 
A bathroom with 0 water and 0 toothpaste. 
A store with 100 water, 99 soap and 97 toothpaste. 
Your home's storage has: 
5 water, 0 toothpaste, 1 soap. 

 Your tasks are: 
Task: Clean 3 dishes 

 These tasks are given in a random order.Your job is to rearrange these tasks in the correct order. You can also add go to, buy, and refill commands.
 Be careful about the given resource contraints and where you are. 
 You can add refill, buy, and go to commands at a cost. 
For example, the costs of buying one item are proportional to real world costs.
 
 You can buy items from the store to restock your pantry by saying: "buy <amount> <item>" You must be in the store to buy items
You can refill any room with items in your storage by saying: "refill <room_name> with <amount> <item>" You must be in the storage to refill items
Remember to say 

'Success'